In [ ]:
%matplotlib qt5
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 
stacks_ssb_2525 = ['/home/somar/Desktop/2025/Data for publication/ssb images/Sample 2525/09-04 After laser before heating/stacks.h5']


stacks_ssb_2344 = ['/home/somar/Desktop/2025/Data for publication/ssb images/Sample 2344/after pt evaporation/stacks.h5']

stack_2525 = stacks_ssb_2525[0]
stack_2344 = stacks_ssb_2344[0]

from imageanalysis.calibrated_images_class import CalibratedImages
calibrated_images = CalibratedImages(stack_2344, font_path=font_path)